# Mental health counseling ChatGPT Clone from Scratch

### Task 1: Import the Libraries

In [2]:
# Import the libraries here
import os as os
from openai import OpenAI
from datasets import load_dataset
from enum import Enum
import random as random
import json as json


/home/postgres/code/educative/mental-health-counseling-bot/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Task 2: Create Classes for the Dataset

In [3]:
# Define the RoleType Enum here
class RoleType(Enum):
    USER = 'user'
    SYSTEM = 'system'
    ASSISTANT = 'assistant'

In [4]:
# Define the Role class here
class Role(object):
    def __init__(self, role_type:RoleType, content):
        self.role = role_type.value
        self.content = content
        self.value = {'role': self.role, 'content':self.content } 

In [5]:
# Define the message class here
class Message(object):
    def __init__(self, user_content, system_content, assistant_content):
        self.user_role = Role(role_type=RoleType.USER , content=user_content)
        self.system_role = Role(role_type=RoleType.SYSTEM , content=system_content)
        self.assistant_role = Role(role_type=RoleType.ASSISTANT, content=assistant_content)
        self.message = {'messages':[self.system_role.value, self.user_role.value, self.assistant_role.value, ]} 

### Task 3: Load, Explore, and Store the Data

In [6]:
# Load the dataset 
dataset = load_dataset(
    "Amod/mental_health_counseling_conversations",
    data_files = "combined_dataset.json",
    split = "train"
)

In [7]:
# Create a sample Message object
context = dataset[152]['Context']
response = dataset[152]['Response']
system_content = "You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."
message_obj = Message(user_content=context, system_content=system_content, assistant_content = response )

print(message_obj.message)

{'messages': [{'role': 'system', 'content': 'You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user’s emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It’s important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsib

In [8]:
#  Create the train_dataset variable
# Sample 100 items from the 'train' split
sampled_dataset = random.choices(dataset, k=100)
train_dataset = []

# Print the sampled data to verify
print(sampled_dataset[1])

for row in sampled_dataset:
    message_obj = Message(user_content=row['Context'], system_content=system_content, assistant_content=row['Response'])
    train_dataset.append(message_obj.message)

print(train_dataset[1])

{'Context': 'They discontinued treatment for no apparent reason after they thought to diagnose me with schizophrenia.', 'Response': "The general rule against medical abandonment\xa0says thst whenever healthcare providers discontinue treatment with you when you still need it, they are legally required to make sure that you have access to another provider of equal or higher ability.In this case, if your treatment team decided to discontinue your treatment based on a diagnosis of schizophrenia, they still will likely be required to provide you with a referral to someone who can provide you with treatment. If you haven't already done so, I'd recommend that you explicitly ask for a referral."}
{'messages': [{'role': 'system', 'content': 'You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit acti

In [9]:
# Save data in JSONl format 
def save_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for row in data:
            line = json.dumps(row)
            file.write(line + '\n')


# Store the data in JSONL format
training_data_path = 'data/train.jsonl'
save_to_jsonl(train_dataset[:-5], training_data_path)

validation_data_path = 'data/validation.jsonl'
save_to_jsonl(train_dataset[-5:], validation_data_path)

### Task 4: Fine-Tune the Model

In [10]:
# Load the training and validation files  
# Load the training and validation files
training_data = open(training_data_path, "rb")
validation_data = open(validation_data_path, "rb")


In [ ]:
# Add OpenAI api_key

api_key = ""
client = OpenAI(api_key=api_key)

In [12]:
# Upload the training and validation files
training_response = client.files.create(file=training_data, purpose="fine-tune")
training_file_id = training_response.id

validation_response = client.files.create(file=validation_data, purpose="fine-tune")
validation_file_id = validation_response.id

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-3HUaff5QbDKNiWVusjKKhA
Validation file id: file-684ZhN3gYwdbzDAymBRiEC


In [19]:
# Create a fine-tuning job
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-4.1-2025-04-14",
    suffix="my-test-model",
    validation_file=validation_file_id
)

job_id = response.id

print(response)

FineTuningJob(id='ftjob-NQjnMiWMUVbUnqccfHwqzoC6', created_at=1768569191, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4.1-2025-04-14', object='fine_tuning.job', organization_id='org-CouWqMcMpoxck15Z7lwN5qYH', result_files=[], seed=1599211923, status='validating_files', trained_tokens=None, training_file='file-3HUaff5QbDKNiWVusjKKhA', validation_file='file-684ZhN3gYwdbzDAymBRiEC', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix='my-test-model', usage_metrics=None, shared_with_openai=False, eval_id=None)


In [20]:
# Retrieve the job status
job_id = response.id

job_status = client.fine_tuning.jobs.retrieve(job_id)
print(job_status)

FineTuningJob(id='ftjob-NQjnMiWMUVbUnqccfHwqzoC6', created_at=1768569191, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4.1-2025-04-14', object='fine_tuning.job', organization_id='org-CouWqMcMpoxck15Z7lwN5qYH', result_files=[], seed=1599211923, status='validating_files', trained_tokens=None, training_file='file-3HUaff5QbDKNiWVusjKKhA', validation_file='file-684ZhN3gYwdbzDAymBRiEC', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix='my-test-model', usage_metrics=None, shared_with_openai=False, eval_id=None)


### Task 5: Test the Fine-Tuned Model

In [21]:
# Create and store message dictionaries
system_message = """You serve as a supportive and honest psychology and psychotherapy assistant. Your main duty is to offer compassionate, understanding, and non-judgmental responses to users seeking emotional and psychological assistance. Respond with empathy and exhibit active listening skills. Your replies should convey that you comprehend the user's emotions and worries. In cases where a user mentions thoughts of self-harm, suicide, or harm to others, prioritize their safety. Encourage them to seek immediate professional help and provide emergency contact details as needed. It's important to note that you are not a licensed medical professional. Refrain from diagnosing or prescribing treatments. Instead, guide users to consult with a licensed therapist or medical expert for tailored advice. Never store or disclose any personal information shared by users. Uphold their privacy at all times. Avoid taking sides or expressing personal viewpoints. Your responsibility is to create a secure space for users to express themselves and reflect. Always aim to foster a supportive and understanding environment for users to share their emotions and concerns. Above all, prioritize their well-being and safety."""

messages = []
messages.append({"role": "system", "content": system_message})
user_message = "Every winter I find myself getting sad because of the weather. How can I fight this?"
messages.append({"role": "user", "content": user_message})

In [22]:
# Test the fine-tuned chat completion model
completion = client.chat.completions.create(
    model= response.model,
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content='Thank you for sharing how you’re feeling—it takes courage to talk about these things. Many people notice changes in their mood when the seasons shift, especially during the winter months. The shorter days, colder weather, and less sunlight can all impact our emotional well-being. What you’re experiencing could be related to something called Seasonal Affective Disorder (SAD), but you’re not alone in feeling this way.\n\nHere are a few gentle ideas that might help manage the sadness you feel during winter:\n\n1. **Spend Time in Natural Light:** Whenever possible, try to get outside during daylight hours. Even a short walk during the brightest part of the day can be uplifting.\n   \n2. **Stay Connected:** Reaching out to friends or loved ones, even virtually, can make a big difference. Social connection often helps reduce feelings of sadness or isolation.\n\n3. **Regular Routine:** Keeping a consistent daily routine, especially with sleep and meals, can brin

In [23]:
# Get and compare the output of the gpt-4.1-2025-04-14 chat completion model
completion = client.chat.completions.create(
    model= "gpt-4.1-2025-04-14",
    messages=messages
)
print(completion.choices[0].message)

ChatCompletionMessage(content="I'm really glad you reached out and shared how you’re feeling. It’s quite common for people to notice a dip in their mood during the winter months, and you’re definitely not alone in experiencing this. The lack of sunlight, shorter days, and colder weather can all contribute to feelings of sadness or low energy, sometimes referred to as the “winter blues.”\n\nYou mentioned wanting to find ways to fight these feelings, which shows a lot of strength and self-awareness. Here are some gentle suggestions that might help:\n\n1. **Try to get as much natural light as possible:** Even on cloudy days, spending time outside or near windows can help. Some people find that using a light therapy lamp in the mornings makes a difference.\n\n2. **Stay active:** Movement, even light exercise like walking or gentle stretching, can boost your mood by releasing endorphins.\n\n3. **Connect with others:** It can be tempting to withdraw, but keeping in touch with friends or fami

# Comparison between the chat completions of the Fine-tuned model and GPT-4.1-2025-04-14

- Tone: Both are empathetic and validating; Fine-tuned model is slightly more “therapist/psychoeducation” styled, GPT-4.1-2025-04-14 is a bit more conversational and “peer-support” toned.
- Specificity: Fine-tuned model is more detailed and structured (more explanation + longer list + more nuance). GPT-4.1-2025-04-14 is slightly shorter and more general.
- Clinical framing: Fine-tuned model explicitly names Seasonal Affective Disorder (SAD) (“could be related…”). GPT-4.1-2025-04-14 avoids labeling/diagnostic language (“winter blues” / “sometimes referred to…”).
- Advice overlap: Both recommend light exposure, routine, movement, social connection, self-compassion, and professional help if it worsens. Fine-tuned model adds a bit more detail around light therapy and “functioning/hopelessness” as escalation cues.
- Closing question: Fine-tuned model asks about past coping + what to try this year (more reflective). GPT-4.1-2025-04-14 asks what winter affects most / which situations are tough (more exploratory, simpler).